In [1]:
import pandas as pd
import numpy as np

### AMT output -> stan input

In [26]:
df1 = pd.read_csv('data/pilot2_1.csv')
df2 = pd.read_csv('data/pilot2_2.csv')
df3 = pd.read_csv('data/pilot2_3.csv')
df = pd.concat([df1, df2, df3])

In [27]:
questions = pd.read_csv('data/questions_update.csv', index_col=0)

In [28]:
df = pd.concat([df1, df2, df3])

In [29]:
cols = []
for i in range(16):
    cols.extend(['Answer.'+str(i+1)+"-" + str(j+1)+"-Q" for j in range(12)])
    cols.extend(['Answer.'+str(i+1)+'-SID'])

In [30]:
tmp = df.iloc[0]
sid_to_topic = {"1":1, "8":2, "15":3, "20":4}

In [31]:
q_ids = [[],[],[],[]]
for i in range(16):
    question_ids = set(list(tmp[['Answer.'+str(i+1)+"-" + str(j+1)+"-Q" for j in range(12)]]))
    topic = sid_to_topic[str(tmp['Answer.'+str(i+1)+'-SID'])]
    q_ids[topic-1].append(question_ids)

In [32]:
def assign_group_id(row, ps):
    id_ = 1
    for i in range(4):
        for j in range(4):
            if row['Answer.'+str(ps+1)+'-'+str(i+1)+'-Q'] in q_ids[i][j]:
                return id_
            id_ +=1
    assert False

In [33]:
problems = pd.DataFrame()
for i in range(16):
    problems['PID_'+str(i+1)] = df.apply(assign_group_id, args=(i, ), axis=1)

In [34]:
def compute_scores(row, ps):
    num_correct = 1 # start at 1 since you can't have a score of 0
    for i in range(12):
        ans = row['Answer.'+str(ps+1)+'-'+str(i+1)+'-A']
        correct_ans = questions[questions['QID']==row['Answer.'+str(ps+1)+'-'+str(i+1)+'-Q']]['CORRECT'].iloc[0]
        if ans==correct_ans:
            num_correct+=1
    return num_correct

In [35]:
for i in range(16):
    df['Y_'+str(i+1)] = df.apply(compute_scores, args=(i, ), axis=1)

In [36]:
def get_topic(row, ps_ind):
    SID = row['Answer.'+str(ps_ind+1)+'-SID']
    return sid_to_topic[str(SID)]

In [37]:
topics = pd.DataFrame()
for i in range(16):
    topics['T_'+str(i+1)] = df.apply(get_topic, args=(i, ), axis=1)

In [38]:
p_order = []
for row in problems.iterrows():
    order = {}
    counter = 0
    for val in row[1]:
        order[counter] = val
        counter += 1
    p_order.append(order)

In [19]:
new_rows = []
cols = range(16)
for row, ind in zip(df.iterrows(), range(len(df))):
    sorted_inds = sorted(cols, key=lambda x:p_order[ind][x])
    sorted_cols = ["Y_"+str(i+1) for i in sorted_inds]
    new_row = list(row[1][sorted_cols])
    new_rows.append(new_row)

In [40]:
self_assessed_scores = []
cols = range(16)
for row, ind in zip(df.iterrows(), range(len(df))):
    sorted_inds = sorted(cols, key=lambda x:p_order[ind][x])
    sorted_cols = ["Answer."+ str(i+1)+"-Conf" for i in sorted_inds]
    new_row = list(row[1][sorted_cols])
    self_assessed_scores.append(new_row)

In [18]:
scores_sorted = pd.DataFrame(new_rows)
scores_sorted.to_csv('data_pilot.csv')

In [41]:
self_scores_sorted = pd.DataFrame(self_assessed_scores)
self_scores_sorted.to_csv('self_data_pilot.csv')

### parameter recovery

In [3]:
df = pd.read_csv('md_fit.csv', index_col=0)

In [5]:
ds = [ 'd_true1',
 'd_true2',
 'd_true3',
 'd_true4',
 'd_true5',
 'd_true6',
 'd_true7',
 'd_true8',
 'd_true9',
 'd_true10',
 'd_true11',
 'd_true12',
 'd_true13',
 'd_true14',
 'd_true15',
 'd_true16']
for d in ds:
    print(np.mean(df[d]))
    print(np.median(df[d]))

-0.09145312594977544
-0.06853047724614605
0.23551084045521747
0.32566389956430486
0.560680115279519
0.5921257223956156
-0.04467423506303305
-0.005118155139547955
0.032135178482212826
0.05585907458247923
-0.185915424322534
-0.11790841679708189
-0.3049070750069066
-0.25198296683817967
0.7934996937281399
0.8348951594873382
0.06811869057282839
0.04740094374815765
0.21654675864858614
0.2637502508124316
0.5569567098988357
0.6051595504298188
-0.04471868925608077
-0.009908603432932485
-0.029172309389915545
-0.04136120417333493
0.6797126755719034
0.717379283473489
0.09458477501566584
0.12895576578913898
-0.03487840810557874
0.004120732424792817


In [43]:
df = pd.read_csv('md_fit.csv', index_col=0)

In [44]:
ds = [ 'd_true1',
 'd_true2',
 'd_true3',
 'd_true4',
 'd_true5',
 'd_true6',
 'd_true7',
 'd_true8',
 'd_true9',
 'd_true10',
 'd_true11',
 'd_true12',
 'd_true13',
 'd_true14',
 'd_true15',
 'd_true16']
for d in ds:
    print(np.mean(df[d]))
    print(np.median(df[d]))

0.49008939947917407
0.4965682920056226
0.5402075328537843
0.5470465202708841
3.8339017822951136
3.795567735330243
0.48822353657497886
0.4918912830681839
0.5011193340357141
0.5021481298660737
0.5643761904524086
0.570662381018211
0.07290264019698316
0.06620382357573565
0.3414313226604554
0.3463829482905143
0.4916118886498146
0.49820770864122205
0.4506913891387717
0.4553499864442116
0.2746954364146104
0.26756222747108666
0.47832273953971427
0.475541419360319
0.5035555397587462
0.5156419963058172
0.5660928831683103
0.5755397843405587
0.08851879802324797
0.08659154383183584
0.4128554958233478
0.40465666331939026


In [26]:
sum(np.logical_and(df['a_true.1']>2, df['a_true.1']<8))/len(df)

0.4955

In [59]:
df = pd.read_csv('md_fit.csv', index_col=0)

In [60]:
for d in ds:
    print(np.mean(df[d]))
    print(np.median(df[d]))

0.5880791740853022
0.5913722655438982
0.28700376746143025
0.2854561739756598
4.096031838732399
4.060993805879523
0.5007941166669299
0.5034133540238999
0.7427857453451029
0.7439765139730866
0.2949955737244212
0.29449492737456573
0.4882102475194281
0.48970532315379955
0.4241542515726411
0.42498808655027853
0.6163992448461229
0.6199263344840207
0.2506475960659711
0.2505093378619734
0.39764663362047664
0.39879495510106033
0.40059055927412784
0.40173673842033414
0.6105981817170815
0.6116501386732537
0.22954501167635508
0.22967381083052468
0.44210243166023144
0.44166082821337466
0.4508980849234469
0.44963178426525663


### results: pilot study

In [43]:
df = pd.read_csv('md_fit_pilot.csv', index_col=0)

In [41]:
omegas = [['Omega.'+str(i)+"."+str(j) for j in range(1,5)] for i in range(1, 5)]

In [43]:
corr = []
for row in omegas:
    corr_row = []
    for col in row:
        corr_row.append(np.mean(df[col]))
    corr.append(corr_row)

In [44]:
corr

[[1.0, 0.055773325331551986, 0.7812348126269762, 0.5587560136656861],
 [0.055773325331551986, 1.0, 0.09555336311921674, 0.13876450435515975],
 [0.7812348126269762, 0.09555336311921674, 1.0, 0.7012791228586784],
 [0.5587560136656861, 0.13876450435515975, 0.7012791228586784, 1.0]]

In [32]:
art_abs_cols = ['a_true.'+str(i)+'.1' for i in range(1,28)]
games_abs_cols = ['a_true.'+str(i)+'.2' for i in range(1,28)]
cities_abs_cols = ['a_true.'+str(i)+'.3' for i in range(1,28)]
math_abs_cols = ['a_true.'+str(i)+'.4' for i in range(1,28)]

In [33]:
cols = [art_abs_cols, games_abs_cols, cities_abs_cols, math_abs_cols]

In [36]:
for (col, name) in zip(cols, ['art', 'video games', 'cities', 'math']):
    print(name)
    ab = [np.mean(df[c]) for c in col]
    print('mean=', np.mean(ab))
    print('low=', min(ab))
    print('high=', max(ab))

art
mean= 0.028549339498902974
low= -0.9941690279135446
high= 1.3838971420140913
video games
mean= -0.0030438278870406825
low= -0.5574109930758592
high= 0.5607637418206266
cities
mean= 0.03429237155855274
low= -0.7371051950407974
high= 0.8996115160258065
math
mean= 0.15134594527895048
low= -2.1688432945327487
high= 1.8612142148317126


In [47]:
[np.mean(df['d_true' + str(i)]) for i in range(1,17)]

[-0.04652123191129955,
 0.2515353677806564,
 0.8998060647716445,
 0.41414817029793954,
 0.18868837362107427,
 0.7310632564304174,
 -0.015700731476609678,
 -0.3051729911921427,
 -0.14263747259660883,
 -0.07531496580748799,
 0.7261458439212499,
 0.6515728418332006,
 -0.6608798938758984,
 -2.4601896612679703,
 -1.1822759319915577,
 -1.3995727983309483]

### self-assessment

In [78]:
df = pd.read_csv('md_fit_self_pilot.csv', index_col=0)

In [79]:
noise_a_cols = ['noise_a1',
 'noise_a2',
 'noise_a3',
 'noise_a4']
[np.mean(df[n]) for n in noise_a_cols]

[-0.538906852805852,
 -0.07622014778066125,
 0.18770836840678262,
 0.37554504682032175]

In [80]:
params_df = pd.read_csv('md_fit_pilot.csv', index_col=0)

In [84]:
[np.mean(params_df['a_true.'+str(10+1)+'.'+str(j)]) for j in range(1,5)]

[1.2246561243128968,
 -0.12450997659027578,
 0.7296030797146398,
 1.1747181451515734]

In [82]:
perceived_d = [np.mean(df["d."+str(i)]) for i in range(1, 17)]
for ind in [0,4,8,12]:
    print(np.mean(perceived_d[ind:ind+4]))

0.39456382029804654
0.3711494777915038
0.38542289135256685
0.21078218749757138


In [83]:
true_d = [np.mean(params_df['d_true' + str(i)]) for i in range(1,17)]
for ind in [0,4,8,12]:
    print(np.mean(true_d[ind:ind+4]))

0.3797420927347352
0.14971947684568482
0.28994156183758846
-1.4257295713665936
